In [11]:
%matplotlib inline
%load_ext autoreload

%autoreload 2
!pip install ../../tensortrade -U


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processing c:\users\codeninja\dev\tensortrade
  Created wheel for tensortrade: filename=tensortrade-0.0.1a18-cp37-none-any.whl size=55090 sha256=959b747453362430204638a527e8eb47c075fb7a00cfd33c8b214f80485c3131
  Stored in directory: C:\Users\CODENI~1\AppData\Local\Temp\pip-ephem-wheel-cache-3lqa6x9p\wheels\f4\72\37\5f9b89cbcd3ef6d4590747c3505b6ed3c72fec82f02d22c51d
Successfully built tensortrade
  Found existing installation: tensortrade 0.0.1a18
    Uninstalling tensortrade-0.0.1a18:
      Successfully uninstalled tensortrade-0.0.1a18


In [12]:
import os
import sys
import warnings
import numpy

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)
numpy.seterr(divide = 'ignore') 

sys.path.append(os.path.dirname(os.path.abspath('')))

In [13]:
import ccxt

from tensortrade.strategies import TensorforceTradingStrategy

In [14]:
from tensortrade.rewards import SimpleProfitStrategy
from tensortrade.actions import DiscreteActionStrategy
from tensortrade.exchanges.simulated import FBMExchange
from tensortrade.features.stationarity import FractionalDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

normalize = MinMaxNormalizer(inplace=True)
difference = FractionalDifference(difference_order=0.6,
                                  inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize, difference])

reward_strategy = SimpleProfitStrategy()
action_strategy = DiscreteActionStrategy(n_actions=20, instrument_symbol='ETH/BTC')

exchange = FBMExchange(base_instrument='BTC',
                       timeframe='1h',
                       should_pretransform_obs=True)

In [15]:
from tensortrade.environments import TradingEnvironment

network_spec = [
    dict(type='dense', size=128, activation="tanh"),
    dict(type='dense', size=64, activation="tanh"),
    dict(type='dense', size=32, activation="tanh")
]

agent_spec = {
    "type": "ppo",
    "learning_rate": 1e-4,
    "discount": 0.99,
    "likelihood_ratio_clipping": 0.2,
    "estimate_terminal": False,
    "max_episode_timesteps": 2000,
    "network": network_spec,
    "batch_size": 10,
    "update_frequency": "never"
}

environment = TradingEnvironment(exchange=exchange,
                                 action_strategy=action_strategy,
                                 reward_strategy=reward_strategy,
                                 feature_pipeline=feature_pipeline)

strategy = TensorforceTradingStrategy(environment=environment, agent_spec=agent_spec)

In [16]:
performance = strategy.run(episodes=10, evaluation=False)

performance[-5:]

Episodes:   0%|                                | 0/10 [00:00, reward=0.00, ts/ep=0, sec/ep=0.00, ms/ts=0.0, agent=0.0%]

Empty DataFrame
Columns: [open, high, low, close, volume]
Index: []


ValueError: Cannot feed value of shape (1, 0, 5) for Tensor 'agent/state-input:0', which has shape '(?, 5)'

In [ ]:
performance.balance.plot()

In [ ]:
strategy.save_agent(directory='agents')